In [24]:
from mintalib import functions as fx



class PricesMethods:
    _prices = None

    def dispatch_prices(self, func, *args, **kwds):
        return func(self._prices, *args, **kwds)

    def atr(self, period: int = 14):
        kwds = dict(period=period)
        self.dispatch_prices(fx.ATR, **kwds)



class SeriesMethods:
    _series = None

    def dispatch_series(self, func, *args, **kwds):
        return func(self._series, *args, **kwds)

    def ema(self, period: int = 20):
        kwds = dict(period=period)
        self.dispatch_series(fx.EMA, **kwds)

    def sma(self, period: int = 20):
        kwds = dict(period=period)
        self.dispatch_series(fx.SMA, **kwds)


def register_pandas(name="ta"):
    import pandas as pd

    @pd.api.extensions.register_dataframe_accessor(name)
    class PricesAccessor(PricesMethods, SeriesMethods):
        def __init__(self, prices):
            self._validate(prices)
            self._prices = prices

        @staticmethod
        def _validate(prices):
            if "close" not in prices.columns:
                raise AttributeError("Prices must have a 'close' column") 

        @property
        def _series(self):
            return self._prices


    @pd.api.extensions.register_series_accessor(name)
    class SeriesAccessor(SeriesMethods):
        def __init__(self, series):
            self._series = series


register_pandas()


/var/folders/c3/54m9yxns31905dkkhzl115lw0000gn/T/ipykernel_49004/2580453487.py:36: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.PricesAccessor'> under name 'ta' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PricesAccessor(PricesMethods, SeriesMethods):
/var/folders/c3/54m9yxns31905dkkhzl115lw0000gn/T/ipykernel_49004/2580453487.py:52: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.SeriesAccessor'> under name 'ta' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  class SeriesAccessor(SeriesMethods):


In [25]:
import yfinance as yf
from mintalib.indicators import SMA

# fetch prices (eg with yfinance)
prices = yf.Ticker('AAPL').history('5y')

# convert column and index names to lower case
prices = prices.rename(columns=str.lower).rename_axis(index=str.lower)

# compute and append indicators to prices
result = prices.assign(
    sma50 = SMA(50),
)
result

,open,high,low,close,volume,dividends,stock splits,sma50
date,,,,,,,,
2019-12-24 00:00:00-05:00,69.000496,69.048973,68.571502,68.898697,48478800,0.0,0.0,NaN
2019-12-26 00:00:00-05:00,69.032005,70.282638,69.002922,70.265671,93121200,0.0,0.0,NaN
2019-12-27 00:00:00-05:00,70.558945,71.249703,69.831833,70.239014,146266000,0.0,0.0,NaN
2019-12-30 00:00:00-05:00,70.156601,70.939461,69.128952,70.655884,144114400,0.0,0.0,NaN
2019-12-31 00:00:00-05:00,70.270508,71.179398,70.171135,71.172127,100805600,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
2024-12-17 00:00:00-05:00,250.080002,253.830002,249.779999,253.479996,51356400,0.0,0.0,233.345886
2024-12-18 00:00:00-05:00,252.160004,254.279999,247.740005,248.050003,56774100,0.0,0.0,233.796448
2024-12-19 00:00:00-05:00,247.500000,252.000000,247.089996,249.789993,60882300,0.0,0.0,234.206494


In [26]:
prices.ta.atr()

In [27]:
prices.close.ta.sma()

In [28]:
prices.ta.sma(50)

In [29]:
prices.ta.ema()

In [30]:
import inspect
def binding_wrapper(func):
    signature = inspect.signature(func)
    def wrapper(*args, **kwargs):
        bound = signature.bind(*args, **kwargs)
        bound.apply_defaults()
        return bound.arguments
    return wrapper

@binding_wrapper
def myfunc(a=1, b=2):
    return dict(a=a, b=b)

myfunc()

{'a': 1, 'b': 2}